In [1]:
from pathlib import Path

from ray import tune
from ray.air import Checkpoint, session

In [2]:
ray_results = Path("~/ray_results").expanduser()
assert ray_results.is_dir()
last_experiment = max(ray_results.glob('_aux_train_daostack_*'), key=lambda x: x.stat().st_ctime)
print(f"Loading results from {last_experiment}...")
assert last_experiment.is_dir()

def _aux_train_daostack(*args, **kwargs):
    raise NotImplementedError

# TODO: Create an experiment grid or whatever is called
tuner = tune.Tuner.restore(str(last_experiment), _aux_train_daostack)
rg = tuner.get_results()
rg.get_best_result()

Loading results from /home/daviddavo/ray_results/_aux_train_daostack_2023-08-17_14-24-39...


2023-08-17 15:50:14,443	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.
2023-08-17 15:50:17,653	WARNING experiment_analysis.py:916 -- Failed to read the results for 1 trials:
- /home/daviddavo/ray_results/_aux_train_daostack_2023-08-17_14-24-39/_aux_train_daostack_7a9516a0_745_trial_index=4,batch_size=4,conv_layers=3,embedding_dim=80,l2=0.0000,learning_rate=0.0012,max_epoc_2023-08-17_15-49-14


Result(
  metrics={'loss': 0.10872686423784196, 'val_loss': 0.8530769944190979, 'rprec train': 0.7320367097854614, 'rprec test': 0.22052255272865295, 'p@5 train': 0.8942307829856873, 'p@5 test': 0.248076930642128, 'r@5 train': 0.21720772981643677, 'r@5 test': 0.15670859813690186, 'should_checkpoint': True, 'done': True, 'trial_id': '066d560f', 'experiment_tag': '241_trial_index=0,batch_size=4,conv_layers=3,embedding_dim=105,l2=0.0000,learning_rate=0.0017,max_epochs=50'},
  path='/home/daviddavo/ray_results/_aux_train_daostack_2023-08-17_14-24-39/_aux_train_daostack_066d560f_241_trial_index=0,batch_size=4,conv_layers=3,embedding_dim=105,l2=0.0000,learning_rate=0.0017,max_epo_2023-08-17_14-50-59',
  checkpoint=Checkpoint(local_path=/home/daviddavo/ray_results/_aux_train_daostack_2023-08-17_14-24-39/_aux_train_daostack_066d560f_241_trial_index=0,batch_size=4,conv_layers=3,embedding_dim=105,l2=0.0000,learning_rate=0.0017,max_epo_2023-08-17_14-50-59/checkpoint_000049)
)

In [3]:
# Read dataframe and drop some unneeded columns
rdf = rg.get_dataframe('rprec test', 'max').drop(columns=['timestamp', 'node_ip', 'pid', 'hostname', 'time_since_restore', 'iterations_since_restore', 'logdir'])
rdf = rdf.drop(columns=['date'])
# rdf = rdf[rdf['done']]
print(rdf.columns)

for c in ['config/batch_size']: #, 'config/embedding_dim']:
    rdf[c] = 2**rdf[c]

rdf.sort_values('loss').head(5)

2023-08-17 15:50:21,513	WARNING experiment_analysis.py:694 -- Failed to read the config for 1 trials:
- /home/daviddavo/ray_results/_aux_train_daostack_2023-08-17_14-24-39/_aux_train_daostack_7a9516a0_745_trial_index=4,batch_size=4,conv_layers=3,embedding_dim=80,l2=0.0000,learning_rate=0.0012,max_epoc_2023-08-17_15-49-14


Index(['loss', 'val_loss', 'rprec train', 'rprec test', 'p@5 train',
       'p@5 test', 'r@5 train', 'r@5 test', 'time_this_iter_s',
       'should_checkpoint', 'done', 'training_iteration', 'trial_id',
       'time_total_s', 'config/__trial_index__', 'config/batch_size',
       'config/conv_layers', 'config/embedding_dim', 'config/l2',
       'config/learning_rate', 'config/max_epochs'],
      dtype='object')


,loss,val_loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,should_checkpoint,...,training_iteration,trial_id,time_total_s,config/__trial_index__,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,config/max_epochs
471,0.102057,0.852359,0.754251,0.213974,0.900000,0.240385,0.218218,0.135020,0.489816,True,...,50,a80077e6,25.089469,0,256,2,65,1.873735e-09,0.01014,50
331,0.106146,0.857666,0.695261,0.204977,0.861538,0.255769,0.202373,0.139001,3.155101,True,...,50,6d9e28b3,153.993793,0,32,5,55,3.167378e-08,0.00469,50
259,0.106258,0.913202,0.701287,0.203587,0.873077,0.259615,0.215629,0.155976,2.810326,True,...,50,539cae6c,136.346081,2,64,5,175,4.646772e-06,0.00253,50
561,0.106943,0.856024,0.775790,0.210333,0.923077,0.242308,0.229748,0.142838,4.060142,True,...,47,c48cc275,187.852929,0,32,5,435,5.457861e-05,0.00295,50
29,0.107736,0.863567,0.722021,0.203510,0.894231,0.223077,0.220726,0.121135,3.462508,True,...,48,0b1a4784,197.281092,1,32,5,345,1.670515e-05,0.00163,50


In [4]:
# Group by config (except trial index) and do the mean
non_config_columns = ['config/__trial_index__', 'config/max_epochs']
cfg_columns = [c for c in rdf.columns if c.startswith('config/') and c not in non_config_columns]
non_results_cols = ['should_checkpoint', 'done', 'config/__trial_index__'] + non_config_columns
rdfgb = rdf.drop(columns=non_results_cols).groupby(cfg_columns).mean(numeric_only=True).reset_index()
print(len(rdfgb), "configurations were tested")
rdfgb.sort_values('rprec test', ascending=False).head(30)

149 configurations were tested


,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,loss,val_loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,training_iteration,time_total_s
101,64,4,200,8.390419e-04,0.01003,0.170990,0.763743,0.736409,0.216214,0.908462,0.239231,0.229216,0.156154,2.436677,30.0,73.652825
14,16,3,105,1.031174e-05,0.00174,0.116141,0.848857,0.694024,0.211434,0.856538,0.245769,0.205194,0.149718,5.971010,35.6,242.533603
77,32,5,460,1.364939e-04,0.00158,0.122078,0.784717,0.720510,0.209439,0.885385,0.251154,0.216106,0.152304,5.332652,45.2,213.626645
103,64,4,315,4.807041e-04,0.00227,0.134665,0.752983,0.702888,0.209063,0.872308,0.247692,0.212053,0.143924,1.935120,39.4,75.684666
112,64,5,200,4.876183e-07,0.00852,0.129401,0.855096,0.738879,0.208211,0.911154,0.237308,0.228557,0.147229,2.015847,25.0,48.924756
73,32,5,345,1.670515e-05,0.00163,0.113829,0.830618,0.701002,0.207112,0.870769,0.242308,0.209120,0.144016,3.951857,41.6,172.038790
95,64,4,50,2.453462e-05,0.00522,0.117889,0.793560,0.664615,0.206489,0.832308,0.237308,0.193931,0.142587,1.968628,40.4,80.954232
66,32,5,45,2.802851e-09,0.00351,0.124993,0.826314,0.621461,0.205680,0.783846,0.236154,0.176188,0.141547,3.220001,42.4,140.656338
109,64,5,175,4.646772e-06,0.00253,0.120075,0.818807,0.655931,0.205539,0.830385,0.247692,0.192844,0.147237,2.681880,41.6,113.474665
68,32,5,55,3.167378e-08,0.00469,0.118592,0.821016,0.661145,0.205145,0.827692,0.241538,0.194213,0.143571,3.084360,37.2,114.276741
